In [39]:
from docx import Document
from spellchecker import SpellChecker
spell = SpellChecker(language='fr')
document = Document('integrale.docx')

In [9]:
document

In [10]:
document.sections

In [11]:
document.paragraphs

 ...]

In [12]:
document.paragraphs[5].runs


[<docx.text.run.Run at 0x1561537a048>, <docx.text.run.Run at 0x1561537a080>]

In [13]:
#print(document.paragraphs[3].runs[0].font.italic)

In [14]:
dir(document.paragraphs[3].style.paragraph_format)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_element',
 '_line_spacing',
 '_line_spacing_rule',
 '_parent',
 '_tab_stops',
 'alignment',
 'element',
 'first_line_indent',
 'keep_together',
 'keep_with_next',
 'left_indent',
 'line_spacing',
 'line_spacing_rule',
 'page_break_before',
 'part',
 'right_indent',
 'space_after',
 'space_before',
 'tab_stops',
 'widow_control']

In [15]:
def isroman(numeral):
    validRomanNumerals = ["M", "D", "C", "L", "X", "V", "I"]
    for letters in numeral:
        if letters not in validRomanNumerals:
            return False
    return True

ROMAN_CONSTANTS = (
            ( "", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX" ),
            ( "", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC" ),
            ( "", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM" ),
            ( "", "M", "MM", "MMM", "",   "",  "-",  "",    "",     ""   ),
        )

ROMAN_SYMBOL_MAP = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)

CUTOFF = 4000
BIG_DEC = 2900
BIG_ROMAN = "MMCM"
ROMAN_NOUGHT = "nulla"

def digits(num):
    if num < 0:
        raise Exception('range error: negative numbers not supported')
    if num % 1 != 0.0:
        raise Exception('floating point numbers not supported')
    res = []
    while num > 0:
        res.append(num % 10)
        num //= 10
    return res

def toString(num, emptyZero=False):
    if num < CUTOFF:
        digitlist = digits(num)
        if digitlist:
            res = reversed([ ROMAN_CONSTANTS[order][digit] for order, digit in enumerate(digitlist) ])
            return "".join(res)
        else:
            return "" if emptyZero else ROMAN_NOUGHT 
    else:
        if num % 1 != 0.0:
            raise Exception('floating point numbers not supported')
        # For numbers over or equal the CUTOFF, the remainder of division by 2900
        # is represented as above, prepended with the multiples of MMCM (2900 in Roman),
        # which guarantees no more than 3 repetitive Ms.
        return BIG_ROMAN * (num // BIG_DEC) + toString(num % BIG_DEC, emptyZero=True)

def parse(numeral):
    numeral = numeral.upper()
    result = 0
    if numeral == ROMAN_NOUGHT.upper():
        return result
    lastVal = 0
    lastCount = 0
    subtraction = False
    for symbol in numeral[::-1]:
        value = ROMAN_SYMBOL_MAP.get(symbol)
        if not value:
            raise Exception('incorrect symbol')
        if lastVal == 0:
            lastCount = 1
            lastVal = value
        elif lastVal == value:
            lastCount += 1
            # exceptions
        else:
            result += (-1 if subtraction else 1) * lastVal * lastCount
            subtraction = lastVal > value
            lastCount = 1
            lastVal = value
    return result + (-1 if subtraction else 1) * lastVal * lastCount

In [34]:
j=0
res=""
for i in document.paragraphs:
    #res+=i.text
    res+='\n\n'
    a=i.runs
    for k in a:
        if isroman(k.text.replace("FABLE ","")):
            t=parse(k.text.replace("FABLE ",""))
            cont="\\clearpage"
            cont+="\n\\section*{Fable "+str(t)+"}\n\\renewcommand{\currsectiontitle}{Fable "+str(t)+"}"
        else:
            if k.text.isupper():
                tt=spell.correction(k.text.lower())
                cont="\\textbf{"+tt+"}"
            else:
                cont=str(k.text)
                if k.font.italic:
                    cont="\\textit{"+cont+"}"
                if k.font.bold:
                    cont="\\textbf{"+cont+"}"
                cont = cont.replace("\xa0", " ")
        cont=cont.replace(" ?","?")
        cont=cont.replace(" !","!")
        cont=cont.replace(" .",".")
        cont=cont.replace(" :",":")
        cont=cont.replace(" ;",";")
        res+=cont
    j=j+1
    if j>50:
        break


f = open("vlat.txt", "w")
f.write(res)
f.close()

        
res

"\n\n\\textbf{integrale des fables de jean de la fontaine}\n\n\n\\textbf{jean}\n\n\\textbf{de la fontaine}\n\n\\textbf{fables}\n\n\\textbf{livres}\n\n\\clearpage\n\\section*{Fable 0}\n\\renewcommand{\\currsectiontitle}{Fable 0}\n\n\n\\textbf{livre premier}\n\n\\clearpage\n\\section*{Fable 1}\n\\renewcommand{\\currsectiontitle}{Fable 1}\n\n\\textbf{la cigale et la fourmi}\n\nLa Cigale, ayant chanté\n\nTout L'Été,\n\nSe trouva fort dépourvue\n\nQuand la Bise fut venue.\n\nPas un seul petit morceau\n\nDe mouche ou de vermisseau.\n\nElle alla crier famine\n\nChez la Fourmi sa voisine,\n\nLa priant de lui prêter\n\nQuelque grain pour subsister\n\nJusqu'à la saison nouvelle.\n\n« Je vous paierai, lui dit-elle,\n\nAvant l'Août, foi d'animal,\n\nIntérêt et principal. »\n\nLa Fourmi n'est pas prêteuse:\n\nC'est là son moindre défaut.\n\n« Que faisiez-vous au temps chaud?\n\nDit-elle à cette emprunteuse.\n\n- Nuit et jour à tout venant\n\nJe chantais, ne vous déplaise.\n\n- Vous chantiez? j'en s

In [42]:
spell.correction("ane")

'ane'